# Web Scraping usando Python

<p>Si nosotros quisieramos hacer un proyecto de análisis de datos, primero necesitamos un set de datos para trabajar. Por supuesto que puede haber diferentes páginas de donde sacar archivos para usar. Pero qué tal si nosotros conseguimos esos datos minando un sitio web.</p>
<br>
<p>En este notebook de Jupyter, vamos a explorar el proceso para extraer información de MercadoLibre y crear un dataset que más tarde podremos usar en análisis de datos.</p>

## Definición de Web Scraping

De Wikipedia en español <a href='https://es.wikipedia.org/wiki/Web_scraping'>[1]</a>:
<p>"<i>Web scraping</i> o raspado web es una técnica utilizada mediante programas de software para extraer información de sitios web. Usualmente, estos programas simulan la navegación de un humano en la World Wide Web ya sea utilizando el protocolo HTTP manualmente, o incrustando un navegador en una aplicación. </p>

<p>El <i>web scraping</i> [...] se enfoca más en la transformación de datos sin estructura en la web (como el formato HTML) en datos estructurados que pueden ser almacenados y analizados en una base de datos central, en una hoja de cálculo o en alguna otra fuente de almacenamiento. Alguno de los usos del <i>web scraping</i> son la comparación de precios en tiendas, la monitorización de datos relacionados con el clima de cierta región, la detección de cambios en sitios webs y la integración de datos en sitios webs. "</p>

Cabe aclarar que no hay un método universal para 'raspar' un sitio web. La disposición de los datos, cómo están estructurados y los permisos para navegar pueden variar radicalmente de un sitio a otro.

## Objetivos

- Comprender cómo MercadoLibre estructura sus datos en páginas HTML para luego redactar algorítmos que se adapten.

- Extraer los enlaces a todos los monitores de escritorio en MercadoLibre y sus especificaciones técnicas haciendo web scraping responsable, eficiente y tolerante a errores.

## Metodología

Usaremos la técnica del 'HTML Parser' <a href='https://en.wikipedia.org/wiki/Web_scraping#Techniques'>[2]</a> que es escencialmente hacer uso de una librería que analiza el HTML y nos ayudará a buscar etiqueta por etiqueta para extraer lo que contienen tanto dentro de sí como sus atributos.

Extraeremos cada dato programáticamente con nula intervención humana.

## Importamos las librerías necesarias

<p>Estas librerías nos ayudarán a leer y extraer datos de cada HTML.</p>

<p>Breve resumen de cada librería:</p>
<ul>
    <li><b>pandas</b>: Librería especializada en almacenamiento y manejo de set de datos.</li>
    <li><b>bs4</b>: Librería para analizar páginas HTML y para buscar tanto etiquetas como contenido.</li>
    <li><b>requests</b>: Librería para realizar solicitudes a páginas web.</li>
    <li><b>time</b>: Librería necesaria para hacer pausas entre solicitudes.</li>
</ul>

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

## Definimos el User Agent

<p>Es buena práctica (en otros sitios es obligatorio) identificarnos como si fuéramos un navegador para poder hacer solicitudes a su servidor.</p>

In [2]:
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"}

## Recabamos los enlaces de cada artículo de la sección escogida

<p>Entramos a cada página desde la primera hasta la última y capturamos los enlaces de cada artículo. Cada solicitud se realiza con un intervalo de tiempo por respeto al servidor y para evitar sanciones que nos impidan seguir con nuestro cometido.</p>

In [3]:
listaURL = []
pagina = 0
cantidad = 1

while True:   

    url = 'https://listado.mercadolibre.com.ar/computacion/monitores-accesorios/monitores/nuevo/monitor-led_Desde_'+str(cantidad)+'_NoIndex_True'
    
    page = requests.get(url,headers=HEADERS)

    if page.status_code >= 400:
        break
    else:    
        pagina += 1

        soup = BeautifulSoup(page.content,'html.parser')    

        for x in soup.find_all('a',class_ = 'ui-search-item__group__element shops__items-group-details ui-search-link'):

            if 'click1' not in str(x['href']):
                listaURL.append(x['href'])

        cantidad += 50
        time.sleep(4)

        
cantidad = None

print('Páginas visitadas: ', pagina)
print('Artículos recabados: ', str(len(listaURL)))

Páginas visitadas:  38
Artículos recabados:  1824


## Definimos algunas de las cabeceras usadas por MercadoLibre

<p>MercadoLibre deja a disposición de los vendedores más de veinte campos para llenar <b>de manera opcional</b>. Por motivos de simpleza, solo usaremos seis de estos campos más el precio y el link al artículo. Se resalta que los campos se llenan opcionalmente porque la cantidad de datos varía de vendedor a vendedor. Esto se maneja más adelante.</p>

In [4]:
cabeceras = ['Marca','Modelo','Tamaño de la pantalla','Tipo de resolución','Tipo de pantalla',
             'Frecuencia de actualización recomendada','Precio','Link']

## Definimos el DataFrame de Pandas

<p>Usaremos la reconocida librería Pandas para guardar los datos que vayamos extrayendo. Después podremos exportar esto a un .CSV para manipular.</p>

In [5]:
df = pd.DataFrame(columns = ['Marca','Modelo','Tamaño de la pantalla','Tipo de resolución',
                             'Tipo de pantalla','Frecuencia de actualización','Precio','Link'])

## Visitamos cada página para extraer los campos que necesitemos

<p>MercadoLibre ubica todos los campos de información técnica en filas de tablas llamadas 'andes_table__row'. Entonces, traemos todas ellas y, de acuerdo a la cabecera de la fila, ubicamos cada dato en su lugar. Nuevamente, la pausa es necesaria para no inundar al servidor con tantas solicitudes juntas y para evitar una posible sanción a nuestra IP.</p>

In [6]:
for articuloURL in listaURL:    
    timeoutException = False
    specsContent = ['','','','','','',0,'']
    rows = []
    while True:            
        if len(rows)>0:
            break
        else:
            try:
                page = requests.get(articuloURL,headers=HEADERS,timeout=5)
            except:
                timeoutException = True
                print('Request timeout. Going to the next URL...')
                break

            soup = BeautifulSoup(page.text,'html')            
            rows = soup.find_all('tr',class_ = 'andes-table__row ui-vpp-striped-specs__row')
            time.sleep(3)

    if page.status_code == 200 and not timeoutException:

        for x in rows:
            cabecera = str(x.find('th').text.strip())

            if cabecera in cabeceras:
                dato = str(x.find('td').text.strip())
                if cabecera == 'Marca':
                    specsContent[0] = dato
                elif cabecera == 'Modelo':
                    specsContent[1] = dato
                elif cabecera == 'Tamaño de la pantalla':
                    specsContent[2] = dato
                elif cabecera == 'Tipo de resolución':
                    specsContent[3] = dato
                elif cabecera == 'Tipo de pantalla':
                    specsContent[4] = dato
                elif 'Frecuencia de actualización' in cabecera:
                    specsContent[5] = dato

        try:    
            specsContent[6]=int(str(soup.find('span',class_='andes-money-amount__fraction').text.strip()).replace('.',''))
        except:                     
            specsContent[6]= -1
        
        specsContent[7] = str(articuloURL)        
        df.loc[df.shape[0]] = specsContent        
        df.to_csv('Monitores Nuevos - Registros sucios - Mercado Libre Argentina - Agosto 2023.csv')


## Resultado

El CSV se llenó correctamente. Aquí se exhiben los primeros cinco registros:

In [7]:
df.head()

,Marca,Modelo,Tamaño de la pantalla,Tipo de resolución,Tipo de pantalla,Frecuencia de actualización,Precio,Link
0,Philips,271E1SCA/55,"27 """,Full HD,LCD,75 Hz,185999,https://www.mercadolibre.com.ar/monitor-gamer-...
1,Samsung,F24T35,"24 """,Full HD,LED,75 Hz,86999,https://www.mercadolibre.com.ar/monitor-gamer-...
2,Samsung,F22T35,"22 """,Full HD,LED,75 Hz,75999,https://www.mercadolibre.com.ar/monitor-gamer-...
3,Noblex,MK24X7100,"23.8 """,Full HD,LED,75 Hz,66990,https://www.mercadolibre.com.ar/monitor-led-no...
4,Philips,221V8,"21.5 """,Full HD,LCD,75 Hz,51499,https://www.mercadolibre.com.ar/monitor-gamer-...


Se lograron recabar 1824 registros, la misma cantidad de enlaces que había en nuestra lista.

In [8]:
len(df)

1824

## Conclusión

MercadoLibre ha resultado ser un sitio de compras amigable con algoritmos de web scraping, tanto en su estructura de HTML fácil de leer como en sus permisos de navegación.

Por otra parte, la cantidad de campos varía de vendedor a vendedor. No hay consistencia en este aspecto y es posible que algunos registros deban ser eliminados por contener campos vacíos.

## Fin del Web Scraping

<p>Ya con los datos almacenados en un archivo .CSV, en el próximo apartado <a href='https://github.com/cris-andrade-97/MeLiWebScraping/blob/main/Limpieza%20del%20CSV/Limpieza%20del%20CSV.ipynb'>'Limpieza del CSV'</a>, limpiaremos y normalizaremos cada campo.</p>